In [1]:
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
import numpy as np
from keras.layers import Conv2D
from keras.layers import Reshape
from keras.layers import MaxPooling2D
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img, img_to_array
# from keras.models import load_model

In [7]:
### Essas linhas fazem a montagem do Meu Drive via código: ###
# from google.colab import drive
# drive.mount('/content/drive')

# CURRDIR = os.path.dirname(__file__)
CURRDIR = f"CNN_dataset/"
TRAINPATH = os.path.join(CURRDIR, "train/")
FILETESTPATH = os.path.join(CURRDIR, "test/2.png")
MODELFILEPATH = os.path.join(CURRDIR, "weights.hdf5")

In [3]:
K.set_image_data_format('channels_last')

Input e Maxpool não tem hiperparâmetros   -    Função MAXPOOL é reduzir o tamanho da imagem     -    Função SOFTMAX é categorizar nas classes do problema
A cada camada CONV estamos aplicando zero padding   -   Conv +Pool considera-se uma camada apenas   -  Flatten transfere de uma dimensão p/ outra (nesse caso 2D [matriz] para 1D [vetor])
Fully-Connected (FC) é chamada assim pq tem 400 neurônios interconectados c/ todos os 120 da camada seguinte (Tradicional Perceptron Model)
Hiperparâmetros p/ camadas convolucionais = (w filtro x h filtro x filtros camada anterior + 1) * filtros camada atual = (5x5x3+1)*6 = 456 (esse 1 é bias do filtro)
Hiperparâmetros p/ camadas FC = (current layer neurons c * previous layer neurons p)+(1*c).  (120 * 400) + (1*120) = 48120
Nota-se que o número nW x nH diminui a medida que aprofundamos a rede... Porém, o número de filtros aumenta nF

In [27]:
model = Sequential()  # define que o modelo é composto de camadas sequências
# Adicionando camadas
model.add(Reshape((3,100,100), input_shape=(100, 100, 3)))  # define que todas as entradas serão tratadas como imagens 100x100, 1 canal
model.add(Conv2D(16, (3, 4), input_shape=(3, 100, 100), activation='relu', padding='same'))  # define o uso de 20 filtros 5x5, função de ativação Relu
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))  # define um downsize das imagens (50% para cada dimensão)
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))  # define o uso de 10 filtros 3x3, função de ativação Relu
model.add(Flatten())  # transforma a matriz 2D em um único vetor
model.add(Dense(32, activation='relu'))  # conecta todas as saídas do vetor a uma rede FC de 64 neurônios, ativação Relu
model.add(Dense(2, activation='softmax'))  # conecta os 64 neurônios da camada anterior a 2 neurônios (classes possíveis de saída), ativação Softmax (define a % de certeza)

# Compilando o modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # categorial_crossentropy serve para várias classes de saída (se não, seria binary_crossentropy)

In [28]:
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 3, 100, 100)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 100, 16)        19216     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 2, 50, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 2, 50, 8)          1160      
                                                                 
 flatten_2 (Flatten)         (None, 800)               0         
                                                                 
 dense_4 (Dense)             (None, 32)                25632     
                                                      

In [29]:
train_datagen = ImageDataGenerator(rescale=1./255,
    horizontal_flip=True)

X_train = train_datagen.flow_from_directory(
    TRAINPATH,
    target_size=(100, 100),
    batch_size=5,
    color_mode='rgb')

Found 50 images belonging to 2 classes.


In [30]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    MODELFILEPATH,
    monitor='val_accuracy',
    verbose=1,
    mode='max')

callbacks_list = [checkpoint]

In [31]:
model.fit(
    X_train,
    steps_per_epoch=len(X_train),
    epochs=20,
    callbacks=callbacks_list)

Epoch 1/20


10/10 [==============================] - ETA: 0s - loss: 0.6477 - accuracy: 0.6200
Epoch 1: saving model to CNN_dataset\weights.hdf5
10/10 [==============================] - 2s 54ms/step - loss: 0.6477 - accuracy: 0.6200
Epoch 2/20
 8/10 [=======================>......] - ETA: 0s - loss: 0.7578 - accuracy: 0.4500
Epoch 2: saving model to CNN_dataset\weights.hdf5
10/10 [==============================] - 0s 48ms/step - loss: 0.7253 - accuracy: 0.4800
Epoch 3/20
10/10 [==============================] - ETA: 0s - loss: 0.5232 - accuracy: 0.7200
Epoch 3: saving model to CNN_dataset\weights.hdf5
10/10 [==============================] - 1s 53ms/step - loss: 0.5232 - accuracy: 0.7200
Epoch 4/20
 9/10 [==========================>...] - ETA: 0s - loss: 0.4505 - accuracy: 0.8000
Epoch 4: saving model to CNN_dataset\weights.hdf5
10/10 [==============================] - 1s 54ms/step - loss: 0.4407 - accuracy: 0.8000
Epoch 5/20
 9/10 [==========================>...] - ETA: 0s - loss: 0.3132 - accura

In [32]:
# model.save(MODELFILEPATH)

# model = load_model(MODELFILEPATH)

In [33]:
img = load_img(FILETESTPATH, target_size=(100, 100), color_mode='rgb')
y = img_to_array(img)
y = np.expand_dims(y, axis=0)

In [34]:
predict = model.predict(y)
classes = np.argmax(predict, axis=1)

1/1 [==============================] - 0s 131ms/step


In [35]:
if(classes[0]==0): print("\nQUADRADO!")
else: print("\nTRIÂNGULO!")


QUADRADO!


In [36]:
directory = f"CNN_dataset/test"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    img = load_img(f, target_size=(100, 100), color_mode='rgb')
    y = img_to_array(img)
    y = np.expand_dims(y, axis=0)
    predict = model.predict(y)
    classes = np.argmax(predict, axis=1)
    if(classes[0]==0): print(filename + " é um QUADRADO!")
    else: print(filename + " é um TRIÂNGULO!")

1/1 [==============================] - 0s 50ms/step
1.png é um QUADRADO!


1/1 [==============================] - 0s 36ms/step
2.png é um QUADRADO!
1/1 [==============================] - 0s 28ms/step
3.png é um TRIÂNGULO!
1/1 [==============================] - 0s 30ms/step
4.png é um TRIÂNGULO!
1/1 [==============================] - 0s 30ms/step
5.png é um TRIÂNGULO!
1/1 [==============================] - 0s 25ms/step
6.png é um QUADRADO!
